# Paquetes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers
import os

# Parametros

In [2]:
# MODEL

_LANGUAGE_         = 'en'
_PRETRAINED_LM_    = 'vinai/bertweet-base'
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.ParallelConfig()
_MAX_SEQ_LEN_      = 128
_OUTPUT_DIR_       = 'parallel_adapter_checkPoints_en'
_LOGGING_STEPS_    = 50


# TRAIN

_NO_GPUS_          = 1
_BATCH_SIZE_       = 100
_EPOCHS_           = 10
_LEARNING_RATE_    = 1e-4

# Dataset

In [3]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 
                  'male':   1}

varietyEN_dict = {'australia'    : 0,
                  'canada'       : 1,
                  'great britain' : 2,
                  'ireland'      : 3,
                  'new zealand'   : 4,
                  'united states': 5}

varietyES_dict = {'argentina': 0,
                  'chile'    : 1,
                  'colombia' : 2,
                  'mexico'   : 3,
                  'peru'     : 4,
                  'spain'    : 5,
                  'venezuela': 6}    

In [4]:
# SET LANGUAGE DIRECTORY

if _LANGUAGE_ == 'en':
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    variety_dict = varietyES_dict

In [5]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
    
vocab = tokenizer.get_vocab()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# CREATE ONE INSTANCE PER DATA SPLIT

from DatasetPAN17 import BasePAN17, DatasetPAN17

baseTrain = BasePAN17(Dir              = 'data',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_)

baseTest  = BasePAN17(Dir              = 'data',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_)


Reading data...
    Done
Tokenizing...
    Done

Total Instances: 72000


Reading data...
    Done
Tokenizing...
    Done

Total Instances: 48000



In [7]:
# CREATE DATA LOADER FOR EVERY TASK

tasks = ['gender', 'variety']

dataset_dict = {}
for task in tasks:
    dataset_dict[task] = DatasetPAN17(Base_Dataset = baseTrain, label = task)

# Model

In [8]:
from transformers import AutoAdapterModel


model = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaAdapterModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Adapters

In [9]:
num_v           = len(baseTrain.variety_dict)
num_labels_dict = {"gender": 2, "variety": num_v,}

# Add adapters
for task_name in tasks:
    
    model.add_adapter(
        adapter_name = task_name, 
        config       = _ADAPTER_CONFIG_
    )
    
    model.add_classification_head(
        head_name    = task_name,
        num_labels   = num_labels_dict[task_name],
      )

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Training

In [10]:
from Training import train_model_with_adapters

train_model_with_adapters(model         = model,
                          dataset_dict  = dataset_dict,
                          epochs        = _EPOCHS_,
                          batch_size    = _BATCH_SIZE_,
                          no_gpus       = _NO_GPUS_,
                          output_dir    = _OUTPUT_DIR_,
                          logging_steps = _LOGGING_STEPS_,
                          learning_rate = _LEARNING_RATE_)

/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 72000
  Num Epochs = 10
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 7200


Step,Training Loss
50,0.686900
100,0.580700
150,0.561200
200,0.582100
250,0.548500
300,0.537100
350,0.520000
400,0.550300
450,0.515400
500,0.517500


Saving model checkpoint to parallel_adapter_checkPoints_en/gender/checkpoint-500
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-500/ge

Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-3500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoin

Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/gender/checkpoint-6500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkP

Step,Training Loss
50,1.659500
100,1.320300
150,1.158100
200,1.100100
250,1.047400
300,1.070500
350,1.006500
400,1.046600
450,1.000400
500,0.989000


Saving model checkpoint to parallel_adapter_checkPoints_en/variety/checkpoint-500
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-500/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-500/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-500/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-500/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-500/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpo

Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-3500/gender/head_config.json
Module weights saved in parallel_adapter_

Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints_en/variety/checkpoint-6500/gender/head_config.json
Module weights saved in parallel_adapter_

# Testing

In [11]:
from TestingPAN17 import test_model_with_adapters

accuracy = test_model_with_adapters(model, baseTest)

100%|███████████████████████████████████████| 2400/2400 [10:53<00:00,  3.67it/s]


In [12]:
accuracy

{'gender': 0.81375, 'variety': 0.8529166666666667, 'joint': 0.69875}